In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Database.csv to Database.csv


In [ ]:
import pandas as pd

df = pd.read_csv("Database.csv")
df.head()

,is fully prepared and completed all the topics in thesyllabus in detail,encourages participation and discussion in the class,"uses modern teaching aids/gadgets (animation, video, simulation, others) in the class and shares handouts/references/PPT/web-resources/others",relates the course content with the real-world scenarios (applications) that helped to improve my interest and understanding of the subject,inspires me by his/her competency in teaching- learning knowledge (how to teach a subject) by making the class most interesting,communicate the desired knowledge to me by knowing my expectations by adopting learner centric education,is dedicated for teaching and punctual in the class,is a role model and life-long mentor for me,"sets an effective question paper in the examination using the standard principles of honesty, dedication, integrity, and seriousness","conducted periodical assessments as per schedule using methods such as quiz, seminars, assignments, classroom presentation, and group discussions",is fair and unbiased in the entire course evaluation process,Final_Grade,Would you pick this instructor again?
0,4,4,5,1,2,2,4,4,5,2,4,3.80,1
1,3,3,3,3,3,4,3,5,5,5,4,4.45,1
2,1,2,1,2,2,1,2,1,1,2,1,4.00,1
3,2,2,1,1,1,1,2,2,1,1,1,4.10,1
4,1,2,2,1,1,2,2,1,2,1,2,4.10,1


In [ ]:
import itertools
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv("Database.csv")

# ✅ Shuffle the dataset before splitting
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Define Features (X) and Target (y)
X = df.drop(columns=['Final_Grade', 'Would you pick this instructor again?'])
y = df['Would you pick this instructor again?']

# Split dataset (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

# Define all 7 ML Models
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "Support Vector Machines": SVC(kernel='linear', random_state=42),
    "Adaboost Classifier": AdaBoostClassifier(n_estimators=30, random_state=42),
    "MLP Classifier": MLPClassifier(hidden_layer_sizes=(50,), max_iter=200, random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=30, random_state=42),
    "XG Boost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=30, random_state=42)
}

# Define Recursive Feature Elimination (RFE) function
def recursive_feature_elimination(X_train, y_train):
    selected_features = list(X_train.columns)
    prev_accuracy = 0

    while len(selected_features) > 1:
        r_squared_values = {}

        # Compute R-squared values for each feature
        for feature in selected_features:
            lr = LinearRegression()
            lr.fit(X_train[[feature]], y_train)
            r_squared_values[feature] = lr.score(X_train[[feature]], y_train)

        # Identify the feature with the lowest R-squared value
        feature_to_remove = min(r_squared_values, key=r_squared_values.get)
        selected_features.remove(feature_to_remove)

        # Train model without the least important feature
        temp_model = DecisionTreeClassifier()
        temp_model.fit(X_train[selected_features], y_train)
        new_accuracy = temp_model.score(X_train[selected_features], y_train)

        # Stop removing features if accuracy decreases or reaches 100%
        if new_accuracy < prev_accuracy or new_accuracy == 1.0:
            break

        prev_accuracy = new_accuracy

    return selected_features

In [ ]:
from tabulate import tabulate

results = []

# Run 49 model combinations
for model1_name, model2_name in itertools.product(models.keys(), repeat=2):
    start_time = time.time()
    print(f"\nTraining {model1_name} -> {model2_name}")

    # Train First Model
    model1 = models[model1_name]
    model1.fit(X_train, y_train)
    y_train_pred = model1.predict(X_train)
    y_test_pred = model1.predict(X_test)

    # Accuracy after first model
    accuracy_model1 = round(accuracy_score(y_test, y_test_pred) * 100, 2)
    print(f"Accuracy after {model1_name}: {accuracy_model1}%")

    # Apply RFE for First Model
    X_train_copy = X_train.copy()
    X_test_copy = X_test.copy()
    X_train_copy["First_Prediction"] = y_train_pred
    X_test_copy["First_Prediction"] = y_test_pred
    selected_features_model1 = recursive_feature_elimination(X_train_copy, y_train)
    X_train_reduced_model1 = X_train_copy[selected_features_model1]
    X_test_reduced_model1 = X_test_copy[selected_features_model1]

    # Retrain First Model on reduced feature set
    model1.fit(X_train_reduced_model1, y_train)
    y_test_pred_rfe1 = model1.predict(X_test_reduced_model1)
    accuracy_model1_rfe = round(accuracy_score(y_test, y_test_pred_rfe1) * 100, 2)
    print(f"Accuracy after Dynamic RFE for {model1_name}: {accuracy_model1_rfe}%")

    # Train Second Model
    model2 = models[model2_name]
    model2.fit(X_train_reduced_model1, y_train)
    y_test_pred2 = model2.predict(X_test_reduced_model1)
    accuracy_model2 = round(accuracy_score(y_test, y_test_pred2) * 100, 2)
    print(f"Accuracy after {model2_name}: {accuracy_model2}%")

    # Apply RFE for Second Model
    selected_features_model2 = recursive_feature_elimination(X_train_reduced_model1, y_train)
    X_train_reduced_model2 = X_train_reduced_model1[selected_features_model2]
    X_test_reduced_model2 = X_test_reduced_model1[selected_features_model2]

    # Retrain Second Model on reduced feature set
    model2.fit(X_train_reduced_model2, y_train)
    y_test_pred_rfe2 = model2.predict(X_test_reduced_model2)
    accuracy_model2_rfe = round(accuracy_score(y_test, y_test_pred_rfe2) * 100, 2)
    print(f"Accuracy after Dynamic RFE for {model2_name}: {accuracy_model2_rfe}%")

    print(f"Completed {model1_name} -> {model2_name} in {round(time.time() - start_time, 2)} seconds")

    # Store results
    results.append([
        model1_name,
        accuracy_model1,
        accuracy_model1_rfe,
        model2_name,
        accuracy_model2,
        accuracy_model2_rfe
    ])

# Print all results in tabular format
headers = [
    "First Model",
    "Accuracy after First Model (%)",
    "Accuracy after Dynamic RFE for First Model (%)",
    "Second Model",
    "Accuracy after Second Model (%)",
    "Accuracy after Dynamic RFE for Second Model (%)"
]

print(tabulate(results, headers=headers, tablefmt="grid"))

+------------------------------+----------------------------------+--------------------------------------------------+------------------------------+-----------------------------------+---------------------------------------------------+
| First Model                  |   Accuracy after First Model (%) |   Accuracy after Dynamic RFE for First Model (%) | Second Model                 |   Accuracy after Second Model (%) |   Accuracy after Dynamic RFE for Second Model (%) |
+==============================+==================================+==================================================+==============================+===================================+===================================================+
| Decision Tree                |                            67.12 |                                            67.16 | Decision Tree                |                             80.71 |                                             80.71 |
+------------------------------+----------------